In [ ]:
!pip3 install kdtree

In [2]:
import random
import math
from shapely.geometry import Polygon as ShapelyPolygon, Point
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon
import kdtree
from matplotlib.patches import Polygon
from map_generators.radnom_walk import generate_random_walk_map
from map_generators.non_intersecting_polygons import generate_non_intersecting_polygons
from map_generators.rect_with_gap import generate_rects_with_gap
from tests.read_test_map import read_test_map
from visualization.map_visualization import show_map_vectorized
from visualization.rrt_visualization import visualize_rrt
from tests.check_results import construct_path_from_node

# Sampling-Based алгоритм RRT для задачи построения маршрута

In this lab, we will consider the problem of finding a path in continuous space for a point robot. For these purposes, we will use a sample-based approach.

The `Rapidly exploring Random Tree` (`RRT`) algorithm, is based on the idea of using a special data structure for planning in continuous space. In a tree, every node except the root has exactly one parent node. At every step, the algorithm creates a new node and try to connect it to the tree. The aim is to generate a node, which close to the goal configuration.

The stages of `RRT` grow at every algorithm step can be summarized as follows:
1) Make random point x<sub>rand</sub> by sampling in configuration space
2) Find the nearest to x<sub>rand</sub> tree node x<sub>near</sub> 
3) Extend the tree from x<sub>near</sub> in the direction of the point x<sub>rand</sub>, create  x<sub>new</sub>


The main advantages of the sampling-based algorithms are the possibility of using it for problems of finding paths in a continuous space, taking into account geometric (for example, the size or shape of the agent) and kinodynamic constraints. Such algorithms are also can be used to find solutions for high-dimensions problems (e.g. motion planning of robot arm), but in this lab we will consider the basic ideas of the sampled-based approach using the well-known problem of finding a path on a plane. To simplify a number of auxiliary operations, we will use a grid representation of the environment, although this algorithm allows us to operate in a completely continuous space.

# Representation

In [ ]:
obstacles_sample = [[(2, 2), (3, 3), (2, 4), (1, 3)],
             [(5, 5), (6, 7), (7, 6)],
             ]

show_map_vectorized(10, 10, obstacles_sample, show_cords=False, show_obstacles_index=True, show_vertexes=True)

# RRT Algorithm

In [4]:
class RRTNode:
    """
    Represents a search node for RRT algorithm.

    Attributes
    ----------
    state : npt.NDArray
        State in space corresponding to current node.
    parent : RRTNode
        Pointer to the parent node.
    trajectory : npt.NDArray
        Trajectory from parent node state to current state
    g : float | int
        Length of the path to the current node.
    """
    def __init__(self, 
                 state,
                 parent: "RRTNode" = None,
                 trajectory = None,
                 g: float = 0.0):
        self.state = state
        self.parent = parent
        self.traj_from_parent = trajectory
        self.g = g

    def __len__(self):
        """
        The auxiliary method required for the correct operation of the KD-tree
        """
        return len(self.state)

    def __getitem__(self, i):
        """
        The auxiliary method required for the correct operation of the KD-tree
        """
        return self.state[i]

    def __repr__(self):
        return 'Vertex({}, {})'.format(self.state[0], self.state[1])

In [5]:
def in_collision(state, 
                 obstacles):
    """
    Checks the state is in conflict with the static environment.
    """
    pass
    # TODO
    
def metric(state1, state2):
    """
    Metric in the state space. In the current lab, the Euclidean distance between points on the plane is used.
    """
    pass
    # TODO

def in_goal_region(state, 
                   state_goal, 
                   region_size):
    """
    Checks if the given state is close enough to the goal
    """
    pass
    # TODO
    
def traj_with_collision(traj, obstacles):
    """
    Checks that the trajectory does not conflict with obstacles.
    """
    pass
    # TODO

def create_new_state(x_initial, y_initial,
                     x_target, y_target,
                     obstacles, 
                     max_transition):
    """
    Selects a new state by moving an incremental distance from state_initial, in the direction of state_target. 

    This function also runs the validation of the trajectory for conflicting with obstacles.

    Note that this lab considers the case when the maximum transition is limited. 
    The constraint is set as the first element of the *args.
    """
    pass
    # TODO

def create_random_state(map_width, map_height, 
                        goal_bias: float, 
                        i_goal, j_goal,
                        goal_sampling_region: float):
    """
    Generates a random configuration in search space. 
    With probability `goal_bias` generates a state in the neighbourhood (`goal_sampling_region`) of the 2 goal state.
    """
    pass
    # TODO

def extend(tree: kdtree, 
           obstacles, 
           state_random,
           max_transition):
    """
    The key function of the RRT algorithm that builds up the search tree. 
    Note that this lab considers the case when the maximum transition is limited.
    """
    pass
    # TODO

In [6]:
def rrt(map_w: int,
        map_h: int,
        obstacles: list, 
        start_x: float, 
        start_y: float, 
        goal_x: float, 
        goal_y: float, 
        max_transition,
        max_iter = 10000, 
        goal_region = 5,
        goal_bias = 0.02,
        ):
    """
    Implements the RRT search algorithm.

    Parameters
    ----------
    task_map : Map
        The grid or map being searched.
    start_x, start_y : float, float
        Starting coordinates.
    goal_x, goal_y : float, float
        Goal coordinates.
    max_iter : int
        Maximum number of iterations of the algorithm
    goal_region : float
        The size of the neighborhood of the goal state where the goal is considered to have been reached by the sample
    goal_bias : float
        The probability with which the state close to the goal is sampled

    Returns
    -------
    Tuple[bool, Optional[RRTNode], int, Optional[KDNode], List[RRTNode]]
        Tuple containing:
        - A boolean indicating if a path was found.
        - The last node in the found path or None.
        - Number of algorithm iterations.
        - RRT search tree.
        - All nodes that have been added to the tree in the order of appending.
    """
    
    node_start = RRTNode([start_x, start_y], None, None, 0.0)
    state_goal = [goal_x, goal_y]
    tree = kdtree.create([node_start])
    iter = 0
    all_points = [(node_start, 0)]
    
    pass
    # TODO

# Tests

<img src="./media/maps.png" alt="drawing" width="700"/>

# Columns

In [7]:
width, height, start, goal, obstacles = read_test_map("./tests/columns.txt")
found, end_node, number_of_steps, tree, all_points = rrt(width, height, obstacles, *start, *goal, max_transition=7, goal_bias=0.05, goal_region=10)

In [ ]:
animation = visualize_rrt(width, height, *start, *goal, obstacles, all_points, construct_path_from_node(end_node), goal_region=10, iterations=number_of_steps, max_iteration=10_000, path_length=end_node.g)
animation

## Random Obstacles 

In [9]:
width, height, start, goal, obstacles = read_test_map("./tests/rand_polygons.txt")
found, end_node, number_of_steps, tree, all_points = rrt(width, height, obstacles, *start, *goal, max_transition=5, goal_bias=0.05, goal_region=10)

In [ ]:
animation = visualize_rrt(width, height, *start, *goal, obstacles, all_points, construct_path_from_node(end_node), goal_region=10, iterations=number_of_steps, max_iteration=10_000, path_length=round(end_node.g,2))
animation

## Massive Tests

In [ ]:
max_transitions = [3, 7, 15]
goal_regions = [5, 10, 25]
goal_biases = [0.05, 0.10, 0.15]

In [ ]:
width, height, start, goal, obstacles = read_test_map("./tests/columns.txt")
results = []

for max_transition in max_transitions:
    for goal_region in goal_regions:
        for goal_bias in goal_biases:
            found, end_node, number_of_steps, tree, all_points = rrt(width, height, obstacles, *start, *goal, max_transition=max_transition, goal_bias=goal_bias, goal_region=goal_region)
            results.append({"found": found, "number of steps": number_of_steps, "number of nodes": len(all_points), "max transition": max_transition, "goal bias": goal_bias, "goal region": goal_region})

for line in results:
    print("-----------------")
    print(f"Max transition: {line['max transition']}")
    print(f"Goal bias: {line['goal bias']}")
    print(f"Goal region: {line['goal region']}")
    print(f"Found: {line['found']}")
    print(f"Number of steps: {line['number of steps']}")
    print(f"Number of nodes: {line['number of nodes']}")
            